In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import numpy as np
#load pneumoniamnist dataset
pneumoniamnist = np.load('pneumoniamnist.npz')

In [ ]:
pneumoniamnist['train_images'].shape, pneumoniamnist['train_labels'].shape

In [ ]:
(x_train, y_train), (x_test, y_test) = (pneumoniamnist['train_images'], pneumoniamnist['train_labels']), (pneumoniamnist['test_images'], pneumoniamnist['test_labels'])
(x_val, y_val) = (pneumoniamnist['val_images'], pneumoniamnist['val_labels'])

#print(x_train.shape)  # (4708, 28, 28)
#print(y_train.shape)  # (4708, 1)
#print(y_train[40:50])  # class-label

#print(x_test.shape)  # (624, 28, 28)
#print(y_test.shape)  # (624, 1)

In [ ]:

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# rescale value to [0 - 1] from [0 - 255]
x_train /= 255  # rescaling
x_test /= 255   # rescaling

x_val = x_val.astype('float32')/255

In [ ]:
x_train.shape+(1,)

In [ ]:
x_train = np.expand_dims(x_train, axis=3)
print('x_train shape:',x_train.shape)
x_test = np.expand_dims(x_test, axis=3)
print('x_test shape:',x_test.shape)
x_val = np.expand_dims(x_val, axis=3)
print('x_val shape:',x_val.shape)

In [ ]:
num_classes = 2

from tensorflow.keras.utils import to_categorical
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)
y_val_onehot = to_categorical(y_val)

print('y_train_onehot shape:', y_train_onehot.shape)
print('y_test_onehot shape:', y_test_onehot.shape)
print('y_val_onehot shape:', y_val_onehot.shape)

In [ ]:
# Ensure image format: resize + grayscale to RGB
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout,Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


x_train = tf.image.resize(x_train, [96, 96])
x_train = tf.image.grayscale_to_rgb(x_train)
x_val = tf.image.resize(x_val, [96, 96])
x_val = tf.image.grayscale_to_rgb(x_val)
x_test = tf.image.resize(x_test, [96, 96])
x_test = tf.image.grayscale_to_rgb(x_test)

x_train = x_train.numpy()
x_val = x_val.numpy()
x_test = x_test.numpy()

num_classes = y_train_onehot.shape[1]

input = Input(shape=x_train.shape[1:])
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=input, outputs=output)

opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
cnn_history = model.fit(x_train, y_train_onehot,
                  batch_size=8,
                  epochs=10,
                  validation_data=(x_val, y_val_onehot))

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter( y=cnn_history.history['accuracy'],
              name='Train'))
fig.add_trace(go.Scatter( y=cnn_history.history['val_accuracy'],
              name='Valid'))
fig.update_layout(height=500,width=700,
              title='Accuracy for race feature',
              xaxis_title='Epoch',
              yaxis_title='Accuracy')
fig.show()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools

classes = ['normal','pneumonia']
print(classification_report(y_test, y_pred, target_names=classes))

print ("**************************************************************")

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(5,5))
plt.title('confusion matrix')
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

fmt = 'd' #'.2f'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

plt.figure(figsize=(5, 3))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()